In [1]:
from __future__ import division
import vertica_python
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from vertica_python import connect
import StringIO
from StringIO import StringIO
from datetime import date, timedelta as td
import xlsxwriter
import os
import json
import urllib2
import base64
import datetime
import re
import csv
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

In [2]:
import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [30]:
def get_service(api_name, api_version, scope, client_secrets_path):
  """Get a service that communicates to a Google API.

  Args:
    api_name: string The name of the api to connect to.
    api_version: string The api version to connect to.
    scope: A list of strings representing the auth scopes to authorize for the
      connection.
    client_secrets_path: string A path to a valid client secrets file.

  Returns:
    A service that is connected to the specified API.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      client_secrets_path, scope=scope,
      message=tools.message_if_missing(client_secrets_path))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage(api_name + '.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  service = build(api_name, api_version, http=http)

  return service

In [31]:
def get_ga_data_segment(service, course_enrolls_dict):

    out = service.data().ga().get(
        ids='ga:' + '86300562',
        start_date=str(course_enrolls_dict['start_date']),
        end_date=str(course_enrolls_dict['end_date']),
        max_results=10000,
        metrics=','.join(course_enrolls_dict['metric_list']),
        dimensions=','.join(course_enrolls_dict['dimension_list']),
        #filters=';'.join(course_enrolls_dict['filter_list']),
        segment='user::sequence::ga:pagePath=~/course/;->ga:pagePath=~/course_modes/;'
        ).execute()
                            
    col_list = []
                            
    for i in np.arange(0,len(out['columnHeaders'])):
        line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
        col_list.append(line)
    return pd.DataFrame(out['rows'],columns=col_list)


In [32]:
def get_ga_data(service, course_enrolls_dict):

    out = service.data().ga().get(
        ids='ga:' + '86300562',
        start_date=str(course_enrolls_dict['start_date']),
        end_date=str(course_enrolls_dict['end_date']),
        max_results=10000,
        metrics=','.join(course_enrolls_dict['metric_list']),
        dimensions=','.join(course_enrolls_dict['dimension_list']),
        filters=';'.join(course_enrolls_dict['filter_list'])).execute()
                            
    col_list = []
                            
    for i in np.arange(0,len(out['columnHeaders'])):
        line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
        col_list.append(line)
    return pd.DataFrame(out['rows'],columns=col_list)


In [33]:
scope = ['https://www.googleapis.com/auth/analytics.readonly']
service = get_service('analytics', 'v3', scope, 'client_secrets.json')

In [34]:
out = service.data().ga().get(
    ids='ga:' + '86300562',
    start_date=str('2016-09-01'),
    end_date=str('2016-11-23'),
    max_results=10000,
    metrics='ga:sessions,ga:transactions',
    dimensions='ga:adContent',
    filters='ga:campaign==mitx',
samplingLevel='HIGHER_PRECISION').execute()

col_list = []

for i in np.arange(0,len(out['columnHeaders'])):
    line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
    col_list.append(line)

In [35]:
config_dict = {
    'filter_list': [
'ga:source==FairandLovelyFoundation'

],
    'dimension_list': [
'ga:date',
'ga:source',
'ga:eventLabel'
],
    'metric_list': [
'ga:sessions',
'ga:goal4Completions',
'ga:goal9Completions'
],
}

In [63]:
config_dict2 = {
    'filter_list': [
'ga:source==FairandLovelyFoundation'

],
    'dimension_list': [
'ga:date',
#'ga:userGender',
'ga:country'
],
    'metric_list': [
'ga:sessions'
],
}

In [64]:
df = pd.DataFrame()

for i in pd.date_range('2017-01-01', '2017-03-01'):

    out = service.data().ga().get(
        ids='ga:' + '86300562',
        start_date=str(i.date()),
        end_date=str(i.date()),
        max_results=10000,
        metrics=','.join(config_dict2['metric_list']),
        dimensions=','.join(config_dict2['dimension_list']),
        filters=';'.join(config_dict2['filter_list']),
    samplingLevel='HIGHER_PRECISION').execute()

    col_list = []

    for i in np.arange(0,len(out['columnHeaders'])):
        line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
        col_list.append(line)
    try:
        df = df.append(pd.DataFrame(out['rows'],columns=col_list))
    except:
        len(out) == 0

In [65]:
df

,date,country,sessions
0,20170101,India,1
0,20170102,India,1
0,20170103,India,4
0,20170104,India,10
0,20170105,India,24
0,20170106,India,19
1,20170106,United States,1
0,20170107,India,32
0,20170108,India,31
0,20170109,(not set),1


In [66]:
df.to_csv('./fair_and_lovely_demographics_country.csv', encoding='utf-8')

In [14]:
out = service.data().ga().get(
    ids='ga:' + '86300562',
    start_date=str('2017-02-03'),
    end_date=str('2017-02-03'),
    max_results=10000,
    metrics='ga:uniqueEvents',
    dimensions='ga:sourceMedium, ga:eventLabel',
    filters='ga:eventAction==edx.course.enrollment.activated',
samplingLevel='HIGHER_PRECISION').execute()

#col_list = []

# for i in np.arange(0,len(out['columnHeaders'])):
#     line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
#     col_list.append(line)
#     test_df = pd.DataFrame(out['rows'],columns=col_list)

In [15]:
out

{u'columnHeaders': [{u'columnType': u'DIMENSION',
   u'dataType': u'STRING',
   u'name': u'ga:sourceMedium'},
  {u'columnType': u'DIMENSION',
   u'dataType': u'STRING',
   u'name': u'ga:eventLabel'},
  {u'columnType': u'METRIC',
   u'dataType': u'INTEGER',
   u'name': u'ga:uniqueEvents'}],
 u'containsSampledData': True,
 u'id': u'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:86300562&dimensions=ga:sourceMedium,+ga:eventLabel&metrics=ga:uniqueEvents&filters=ga:eventAction%3D%3Dedx.course.enrollment.activated&start-date=2017-02-03&end-date=2017-02-03&max-results=10000',
 u'itemsPerPage': 10000,
 u'kind': u'analytics#gaData',
 u'profileInfo': {u'accountId': u'35248639',
  u'internalWebPropertyId': u'83315850',
  u'profileId': u'86300562',
  u'profileName': u'All Web Site Data',
  u'tableId': u'ga:86300562',
  u'webPropertyId': u'UA-35248639-2'},
 u'query': {u'dimensions': u'ga:sourceMedium, ga:eventLabel',
  u'end-date': u'2017-02-03',
  u'filters': u'ga:eventAction==edx.course.e

In [41]:
out = service.data().ga().get(
    ids='ga:' + '86300562',
    start_date='2017-01-25',
    end_date='2017-01-25',
    max_results=10000,
    metrics='ga:uniquepageViews',
    dimensions='',
    filters='ga:pagePath==/basket/',
    segment='sessions::condition::ga:eventAction==edx.bi.ecommerce.basket.payment_selected;ga:goal10Completions>0;ga:dimension9!=hosted',
    samplingLevel='HIGHER_PRECISION'
    ).execute()

In [42]:
out

{u'columnHeaders': [{u'columnType': u'METRIC',
   u'dataType': u'INTEGER',
   u'name': u'ga:uniquepageViews'}],
 u'containsSampledData': True,
 u'id': u'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:86300562&metrics=ga:uniquepageViews&filters=ga:pagePath%3D%3D/basket/&segment=sessions::condition::ga:eventAction%3D%3Dedx.bi.ecommerce.basket.payment_selected;ga:goal10Completions%3E0;ga:dimension9!%3Dhosted&start-date=2017-01-25&end-date=2017-01-25&max-results=10000',
 u'itemsPerPage': 10000,
 u'kind': u'analytics#gaData',
 u'profileInfo': {u'accountId': u'35248639',
  u'internalWebPropertyId': u'83315850',
  u'profileId': u'86300562',
  u'profileName': u'All Web Site Data',
  u'tableId': u'ga:86300562',
  u'webPropertyId': u'UA-35248639-2'},
 u'query': {u'dimensions': u'',
  u'end-date': u'2017-01-25',
  u'filters': u'ga:pagePath==/basket/',
  u'ids': u'ga:86300562',
  u'max-results': 10000,
  u'metrics': [u'ga:uniquepageViews'],
  u'samplingLevel': u'HIGHER_PRECISION',
  u'segm

In [23]:
out = service.data().ga().get(
    ids='ga:' + '86300562',
    start_date=str('2016-12-10'),
    end_date=str('2016-12-10'),
    max_results=10000,
    metrics='ga:uniqueEvents',
    #dimensions='ga:page',
    filters='ga:pagePath==/business;ga:eventAction==edx.bi.enterprise.form.submitted').execute()

col_list = []

for i in np.arange(0,len(out['columnHeaders'])):
    line = re.sub('ga.', '', out['columnHeaders'][i][u'name'])
    col_list.append(line)

error: [Errno 49] Can't assign requested address

In [64]:
page_path_dict = {
    'filter_list': [
'ga:pagePath=~^/course_modes/',
],
    'dimension_list': [
'ga:date',
'ga:pagePath',
],
    'metric_list': [
'ga:pageviews',
'ga:uniquePageviews'
],
    'start_date': '2016-10-01',
    'end_date': '2016-10-08',
}

In [65]:
df = get_ga_data_segment(service, page_path_dict)#.groupby(['date','eventLabel']).count()

HttpError: <HttpError 400 when requesting https://www.googleapis.com/analytics/v3/data/ga?max-results=10000&dimensions=ga%3Adate%2Cga%3ApagePath&start-date=2016-10-01&end-date=2016-10-08&ids=ga%3A86300562&metrics=ga%3Apageviews%2Cga%3AuniquePageviews&alt=json&segment=user%3A%3Asequence%3A%3Aga%3ApagePath%3D%7E%2Fcourse%2F%3B-%3Ega%3ApagePath%3D%7E%2Fcourse_modes%2F%3B returned "Invalid value 'user::sequence::ga:pagePath=~/course/;->ga:pagePath=~/course_modes/;' for segment parameter.">

In [54]:
df

,date,pagePath,pageviews,uniquePageviews
0,20161001,/course_modes/choose/course-v1:ANUx+ANU-ASTRO1...,28,24
1,20161001,/course_modes/choose/course-v1:ANUx+ANU-ASTRO2...,25,23
2,20161001,/course_modes/choose/course-v1:ANUx+ANU-ASTRO3...,14,14
3,20161001,/course_modes/choose/course-v1:ANUx+ANU-ASTRO4...,19,19
4,20161001,/course_modes/choose/course-v1:ANUx+ANU-Actuar...,17,16
5,20161001,/course_modes/choose/course-v1:ANUx+EBM01x+1T2...,15,13
6,20161001,/course_modes/choose/course-v1:ANUx+EBM02x+2T2...,9,6
7,20161001,/course_modes/choose/course-v1:ANUx+EBM03x+2T2...,7,6
8,20161001,/course_modes/choose/course-v1:ANUx+EBM04x+2T2...,5,3
9,20161001,/course_modes/choose/course-v1:ANUx+EBM05x+3T2...,6,4


In [7]:
course_views_dict = {
    'filter_list': [
'ga:pagePath=~^/course/',
],
    'dimension_list': [
'ga:date',
'ga:pageTitle',
#'ga:pagePath' 
],
    'metric_list': [
'ga:pageviews',
'ga:uniquePageviews'
],
    'start_date': '2016-10-01',
    'end_date': '2016-10-08',
}

In [32]:
test_dict = {
    'filter_list': [
'ga:eventAction==edx.bi.user.discovery.card.click',
        'ga:eventCategory==program',
        'ga:dimension6==true',
        'ga:pagePath==/',
],
    'dimension_list': [
'ga:date','ga:eventLabel','ga:dimension5'
],
    'metric_list': [
'ga:totalEvents','ga:uniqueEvents'
],
    'start_date': '2016-11-10',
    'end_date': '2016-11-15',
}

In [33]:
df = get_ga_data(service, test_dict)#.groupby(['date','eventLabel']).count()

In [39]:
df.groupby(['date','eventLabel','dimension5']).sum()

totalEvents  \
date     eventLabel                                         dimension5               
20161110 Artificial Intelligence [9549586e-0993-4903-913... 0                   66   
                                                            1                  526   
         Human Rights [cf337394-ab9f-4e41-800c-7455f4a4e... 3                   15   
         International Business Management [f59a9d8f-fe0... 2                   58   
         International Hospitality Management [281102cf-... 1                   15   
         Project Management [38dde1fd-e9a4-4f4b-9c87-87a... 3                  372   
         Supply Chain Management [2fc3236d-78a9-45a1-8c0... 0                  328   
         User Experience (UX) Research and Design [10339... 2                  226   
20161111 Artificial Intelligence [9549586e-0993-4903-913... 0                  686   
                                                            1                    7   
         Human Rights [cf337394-ab9f-4e41-800c-7455f4a4e... 3                  109   
         International Business Management [f59a9d8f-fe0... 2                  328   
         International Hospitality Management [281102cf-... 1                  219   
20161112 Artificial Intelligence [9549586e-0993-4903-913... 0                  730   
         Human Rights [cf337394-ab9f-4e41-800c-7455f4a4e... 3                  153   
         International Business Management [f59a9d8f-fe0... 2                  321   
         International Hospitality Management [281102cf-... 1                  190   
         Project Management [38dde1fd-e9a4-4f4b-9c87-87a... 3                    7   
         User Experience (UX) Research and Design [10339... 2                    7   
20161113 Artificial Intelligence [9549586e-0993-4903-913... 0                  708   
         Human Rights [cf337394-ab9f-4e41-800c-7455f4a4e... 3                   95   
         International Business Management [f59a9d8f-fe0... 2                  401   
         International Hospitality Management [281102cf-... 1                  255   
20161114 Artificial Intelligence [9549586e-0993-4903-913... 0                  774   
         Human Rights [cf337394-ab9f-4e41-800c-7455f4a4e... 3                  226   
         International Business Management [f59a9d8f-fe0... 2                  423   
         International Hospitality Management [281102cf-... 1                  248   
20161115 Artificial Intelligence [9549586e-0993-4903-913... 0                  591   
         Biobased Sciences for Sustainability [0a5a283f-... 2                  124   
         Human Rights [cf337394-ab9f-4e41-800c-7455f4a4e... 3                  124   
         International Business Management [f59a9d8f-fe0... 2                  307   
         International Hospitality Management [281102cf-... 1                  255   
         Project Management [38dde1fd-e9a4-4f4b-9c87-87a... 1                  416   
         Supply Chain Management [2fc3236d-78a9-45a1-8c0... 3                  161   
         User Experience (UX) Research and Design [10339... 0                  212   

                                                                       uniqueEvents  
date     eventLabel                                         dimension5               
20161110 Artificial Intelligence [9549586e-0993-4903-913... 0                    58  
                                                            1                   474  
         Human Rights [cf337394-ab9f-4e41-800c-7455f4a4e... 3                    15  
         International Business Management [f59a9d8f-fe0... 2                    58  
         International Hospitality Management [281102cf-... 1                    15  
         Project Management [38dde1fd-e9a4-4f4b-9c87-87a... 3                   350  
         Supply Chain Management [2fc3236d-78a9-45a1-8c0... 0                   307  
         User Experience (UX) Research and Design [10339... 2                   212  
20161111 Artificial Intellig

In [202]:
course_views_df = get_ga_data(service, course_views_dict)

In [204]:
course_views_df.sort_values('uniquePageviews', ascending=False).head()

,date,pageTitle,pageviews,uniquePageviews
6644,20161001,Analytics for the Classroom Teacher | edX,126,98
2558,20161001,2.6 Reference Questions | TOEFLx Courseware | edX,110,98
1165,20161001,1.3. Hacia la sostenibilidad de la región | Ma...,131,97
5414,20161001,A taste of things to come | Think101x Coursewa...,118,97
2623,20161001,2.9 Prose Summary and Fill in a Table Question...,107,97


In [543]:
course_enroll_reg_dict = {
    'filter_list': [
],
    'dimension_list': [
        'ga:eventLabel',
        'ga:campaign',
        'ga:sourceMedium',
        'ga:fullReferrer',
],
    'metric_list': [
        'ga:goal8Completions',
        'ga:goal4Completions',
],
    'start_date': '2016-10-15',
    'end_date': '2016-10-15',
}

In [544]:
course_enrolls_df = get_ga_data(service, course_enroll_reg_dict)

In [545]:
course_enrolls_df#.groupby('eventLabel').sum()

,eventLabel,campaign,sourceMedium,fullReferrer,goal8Completions,goal4Completions
0,BerkeleyX/CS-191x/2013_August,(not set),(direct) / (none),(direct),5,1
1,BerkeleyX/CS-191x/2013_August,(not set),google / organic,google,7,2
2,BerkeleyX/CS-191x/2013_August,(not set),rigetti.com / referral,rigetti.com/resources,1,0
3,BerkeleyX/CS188x_1/1T2013,(not set),(direct) / (none),(direct),7,0
4,BerkeleyX/CS188x_1/1T2013,(not set),ai.berkeley.edu / referral,ai.berkeley.edu/project_instructions.html,1,0
5,BerkeleyX/CS188x_1/1T2013,(not set),aima.cs.berkeley.edu / referral,aima.cs.berkeley.edu/,9,1
6,BerkeleyX/CS188x_1/1T2013,(not set),aima.cs.berkeley.edu / referral,aima.cs.berkeley.edu/index.html,1,1
7,BerkeleyX/CS188x_1/1T2013,(not set),aw / affiliate_partner,aw,2,1
8,BerkeleyX/CS188x_1/1T2013,(not set),baidu / organic,baidu,1,1
9,BerkeleyX/CS188x_1/1T2013,(not set),berkeley.edu / referral,berkeley.edu/academics/online-learning,1,0


In [236]:
len(course_enrolls_df)

3860

In [513]:
course_purchase_dict = {
    'filter_list': [
],
    'dimension_list': [
        'ga:productName',
        'ga:campaign',
        'ga:sourceMedium',
        'ga:fullReferrer',
],
    'metric_list': [
'ga:itemRevenue',
'ga:uniquePurchases'
],
    'start_date': '2016-10-15',
    'end_date': '2016-10-15',
}

In [514]:
course_purchase_df = get_ga_data(service, course_purchase_dict)

In [515]:
course_page_df

,productName,campaign,sourceMedium,fullReferrer,itemRevenue,uniquePurchases
0,course-v1:ACCA+FFA-F3.x+1T2017,(not set),outlook.live.com / referral,outlook.live.com/,89.0,1
1,course-v1:ASUx+AST111x+2167B,(not set),google / organic,google,98.0,2
2,course-v1:ASUx+ENG101x+2171A,(not set),(direct) / (none),(direct),49.0,1
3,course-v1:ASUx+ENG102x+2167B,(not set),google / organic,google,49.0,1
4,course-v1:ASUx+EXW100x+2167B,(not set),(direct) / (none),(direct),98.0,2
5,course-v1:ASUx+EXW100x+2167B,(not set),google / organic,google,49.0,1
6,course-v1:ASUx+EXW100x+2167B,(not set),mg.mail.yahoo.com / referral,mg.mail.yahoo.com/neo/launch,49.0,1
7,course-v1:ASUx+EXW100x+2167B,studentnewsletter-20161004,sailthru / email,sailthru,49.0,3
8,course-v1:ASUx+HST102x+2167B,(not set),google / organic,google,147.0,3
9,course-v1:ASUx+MAT117x+1T2016,(not set),google / organic,google,49.0,1


In [501]:
len(course_page_df)

341

In [649]:
test_dict = {
    'filter_list': [
        'ga:eventAction==edx.bi.user.discovery.card.click',
        'ga:eventCategory==program',
        'ga:dimension6==true',
        'ga:pagePath==/',
],
    'dimension_list': [
        'ga:date',
        'ga:eventLabel',
        'ga:eventCategory',
        #'ga:dimension4',
        'ga:dimension5',
        'ga:dimension6',
        'ga:pagePath'
],
    'metric_list': [
'ga:totalEvents','ga:uniqueEvents'
],
    'start_date': '2016-11-08',
    'end_date': '2016-11-08',
}

In [650]:
test_df = get_ga_data(service, test_dict)

In [651]:
test_df[(test_df.eventLabel=='Supply Chain Management [2fc3236d-78a9-45a1-8c0c-fc290e74259e]')
       & (test_df.pagePath=='/')]

,date,eventLabel,eventCategory,dimension5,dimension6,pagePath,totalEvents,uniqueEvents
1521,20161108,Supply Chain Management [2fc3236d-78a9-45a1-8c...,program,0,true,/,538,491


In [583]:
test_df#['dimension5']#.unique()

,date,eventLabel,eventCategory,dimension5,pagePath,totalEvents,uniqueEvents
0,20161107,(not set),search,0,/,3,3
1,20161107,(not set),search,0,/course,1,1
2,20161107,(not set),search,0,/course?course=all,1,1
3,20161107,(not set),search,0,/course?program=all,1,1
4,20161107,(not set),search,1,/,3,3
5,20161107,(not set),search,11,/,1,1
6,20161107,(not set),search,3,/,3,3
7,20161107,(not set),search,3,/course,3,3
8,20161107,(not set),search,3,/course?course=all,3,3
9,20161107,(not set),search,4,/course,1,1
